In [1]:
import sys
sys.path.insert(0, '..')  # Add the parent directory (project root directory) to the Python path

In [2]:
from utils.processing import filtered_df

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('../data/bond_etf_yield.csv', index_col=0)
df.to_csv('C:/Code/Python/cape/django_apps/cape/data/bond_etf_yield.csv')

In [5]:
df.sort_values(by='Yield to Maturity', ascending=False).dropna().head(10)

,Name,Yield to Maturity,Downside Volatility,Sortino Ratio,Sortino Ratio Z-Score,P/E Ratio,Date
Ticker,,,,,,,
BKLN,Invesco Senior Loan ETF,0.0932,0.0344,1.38,3.09,10.73,04-05-2024
JBBB,B-BBB CLO ETF,0.0902,0.0445,1.00,2.14,11.09,03-28-2024
SRLN,SPDR® Blackstone Senior Loan ETF,0.0884,0.0360,1.19,2.62,11.31,03-28-2024
HYEM,VanEck Emerging Markets High Yield Bond ETF,0.0867,0.0856,0.48,0.85,11.53,04-10-2024
BRLN,BlackRock Floating Rate Loan ETF,0.0867,0.0353,1.16,2.54,11.53,03-28-2024
HYGV,FLEXSHARES HIGH YIELD VALUE-SCORED BOND INDEX ...,0.0858,0.0511,0.79,1.62,11.66,03-28-2024
HYGH,iShares Interest Rate Hedged High Yield Bond ETF,0.0854,0.0515,0.77,1.57,11.71,03-28-2024
EMHY,iShares J.P. Morgan EM High Yield Bond ETF,0.0850,0.0629,0.63,1.22,11.76,03-28-2024
FTSL,First Trust Senior Loan Fund,0.0835,0.0334,1.13,2.47,11.98,03-28-2024


In [6]:
df.sort_values(by='Sortino Ratio', ascending=False).dropna().head(10)

,Name,Yield to Maturity,Downside Volatility,Sortino Ratio,Sortino Ratio Z-Score,P/E Ratio,Date
Ticker,,,,,,,
BKLN,Invesco Senior Loan ETF,0.0932,0.0344,1.38,3.09,10.73,04-05-2024
SRLN,SPDR® Blackstone Senior Loan ETF,0.0884,0.0360,1.19,2.62,11.31,03-28-2024
JSI,Securitized Income ETF,0.0815,0.0309,1.16,2.54,12.27,03-28-2024
BRLN,BlackRock Floating Rate Loan ETF,0.0867,0.0353,1.16,2.54,11.53,03-28-2024
FTSL,First Trust Senior Loan Fund,0.0835,0.0334,1.13,2.47,11.98,03-28-2024
PGHY,Invesco Global ex-US High Yield Corporate Bond...,0.0831,0.0361,1.04,2.24,12.03,04-05-2024
IHYF,Invesco High Yield Bond Factor ETF,0.0807,0.0346,1.01,2.17,12.39,04-05-2024
JBBB,B-BBB CLO ETF,0.0902,0.0445,1.00,2.14,11.09,03-28-2024
CLOA,BlackRock AAA CLO ETF,0.0686,0.0244,0.94,1.99,14.58,03-28-2024


In [7]:
df.loc['BRLN']

Name                     BlackRock Floating Rate Loan ETF
Yield to Maturity                                  0.0867
Downside Volatility                                0.0353
Sortino Ratio                                        1.16
Sortino Ratio Z-Score                                2.54
P/E Ratio                                           11.53
Date                                           03-28-2024
Name: BRLN, dtype: object

In [8]:
target_rate = 5.5  # percent

# Assuming filtered_df function is defined elsewhere in your code
selected_etfs = filtered_df(
    df, 
    min_zscore=1,
    file_path='C:/Code/Python/work/nrcapital/data/bond_etf_buy_list.csv', 
    return_df=True
)

# Convert 'Yield to Maturity' column to float after replacing '%' with ''
selected_etfs['Yield to Maturity'] = selected_etfs['Yield to Maturity'].str.replace('%', '').astype(float)

# Filter ETFs based on the target rate
selected_etfs = selected_etfs[selected_etfs['Yield to Maturity'] >= target_rate].sort_values(by='Sortino Ratio', ascending=False)
selected_etfs['Yield to Maturity'] = selected_etfs['Yield to Maturity'].astype(str) + '%'

selected_etfs

,Name,Yield to Maturity,Downside Volatility,Sortino Ratio,Sortino Ratio Z-Score,P/E Ratio,Date
Ticker,,,,,,,
BKLN,Invesco Senior Loan ETF,9.32%,3.44%,1.38,3.09,10.73,04-05-2024
SRLN,SPDR® Blackstone Senior Loan ETF,8.84%,3.60%,1.19,2.62,11.31,03-28-2024
BRLN,BlackRock Floating Rate Loan ETF,8.67%,3.53%,1.16,2.54,11.53,03-28-2024
JSI,Securitized Income ETF,8.15%,3.09%,1.16,2.54,12.27,03-28-2024
FTSL,First Trust Senior Loan Fund,8.35%,3.34%,1.13,2.47,11.98,03-28-2024
PGHY,Invesco Global ex-US High Yield Corporate Bond...,8.31%,3.61%,1.04,2.24,12.03,04-05-2024
IHYF,Invesco High Yield Bond Factor ETF,8.07%,3.46%,1.01,2.17,12.39,04-05-2024
JBBB,B-BBB CLO ETF,9.02%,4.45%,1.00,2.14,11.09,03-28-2024
BSJP,Invesco BulletShares 2025 High Yield Corporate...,7.03%,2.64%,0.94,1.99,14.22,04-05-2024


## Filter By Dates

In [9]:
from datetime import datetime

df_ = df.copy()

# Convert the Date column to datetime
df_['Date'] = pd.to_datetime(df_['Date'])

# Find the current month
current_month = datetime.now().month

# Filter the rows where the Date column's month is before the current month
filtered_df = df_[df_['Date'].dt.month != current_month]

filtered_df

,Name,Yield to Maturity,Downside Volatility,Sortino Ratio,Sortino Ratio Z-Score,P/E Ratio,Date
Ticker,,,,,,,
AGG,iShares Core U.S. Aggregate Bond ETF,0.0484,0.0544,0.05,-0.22,20.66,2024-03-28
TLT,iShares 20+ Year Treasury Bond ETF,0.0444,0.1081,-0.01,-0.37,22.52,2024-03-28
MUB,iShares National Muni Bond ETF,0.0340,0.0307,-0.38,-1.29,29.41,2024-03-28
LQD,iShares iBoxx $ Investment Grade Corporate Bon...,0.0526,0.0677,0.10,-0.09,19.01,2024-03-28
MBB,iShares MBS ETF,0.0501,0.0940,0.05,-0.22,19.96,2024-03-28
...,...,...,...,...,...,...,...
MBSD,FLEXSHARES DISCIPLINED DURATION MBS INDEX FUND,0.0491,0.0574,0.06,-0.19,20.37,2024-03-28
RAVI,FLEXSHARES ULTRA-SHORT INCOME FUND,0.0571,0.0307,0.37,0.58,17.51,2024-03-28
SKOR,FLEXSHARES CREDIT-SCORED US CORPORATE BOND IND...,0.0531,0.0481,0.16,0.06,18.83,2024-03-28


In [10]:
filtered_df.sort_values(by='Date').head(10)

,Name,Yield to Maturity,Downside Volatility,Sortino Ratio,Sortino Ratio Z-Score,P/E Ratio,Date
Ticker,,,,,,,
SCHR,Schwab Intermediate-Term U.S. Treasury ETF,0.0388,0.0579,-0.12,-0.64,25.77,2023-12-31
SCYB,Schwab High Yield Bond ETF,0.0765,0.0434,0.71,1.42,13.07,2023-12-31
SCHI,Schwab 5-10 Year Corporate Bond ETF,0.0500,0.0662,0.07,-0.17,20.00,2023-12-31
SCHJ,Schwab 1-5 Year Corporate Bond ETF,0.0504,0.0252,0.19,0.13,19.84,2023-12-31
SCHO,Schwab Short-Term U.S. Treasury ETF,0.0430,0.0187,-0.14,-0.69,23.26,2023-12-31
SCHZ,Schwab U.S. Aggregate Bond ETF,0.0453,0.0452,-0.01,-0.37,22.08,2023-12-31
SCHP,Schwab U.S. TIPS ETF,0.0397,0.0451,-0.13,-0.67,25.19,2023-12-31
SCMB,Schwab Municipal Bond ETF,0.0366,0.0275,-0.33,-1.16,27.32,2023-12-31
SCHQ,Schwab Long-Term U.S. Treasury ETF,0.0414,0.0988,-0.04,-0.44,24.15,2023-12-31
